In [1]:
#%pip install ipyleaflet requests openai

In [2]:
import requests
import pandas as pd
from IPython.display import display
from ipyleaflet import Map as LeafletMap, DrawControl
from transformers import pipeline

/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/sophiakim/miniconda3/envs/virtual_env/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feed

In [3]:
# Once data has been processed, we check if there are any invalid data (i.e. marker is on land)
def is_valid_data(data):
    hourly_data = data['hourly']

    for hourly_entry in hourly_data.items():
        values = hourly_entry[1]
        if all(value is None for value in values):
            return False

    return True

In [4]:
# Initialize llm with transformers library
llm = pipeline("summarization", model="t5-large")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [5]:
# Clean the data before passing to llm
def clean_data(data):
    df = pd.DataFrame(data['hourly'])
    df.columns = ['time', 'wind_wave_height', 'wind_wave_direction', 'wind_wave_period']
    
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        df[col] = df[col].astype(str)
    
    clean_data = df.values.tolist()
    return clean_data

In [8]:
# Event handler when user pins marker on map
def handle_marker(event, action, geo_json):
    coordinates = geo_json['geometry']['coordinates']
    longitude, latitude = str(coordinates[0]), str(coordinates[1])

    # Fetch sea conditions from Open-Meteo API
    url = "https://marine-api.open-meteo.com/v1/marine?latitude=" + latitude + "&longitude=" + longitude + "&hourly=wind_wave_height,wind_wave_direction,wind_wave_period&timezone=auto&forecast_days=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if is_valid_data(data):
            # print(data)
            clean_summary = clean_data(data)
            summary = llm(clean_summary, max_length=6, min_length=1, do_sample=False)
            print(summary)
        else:
            print("Please try again")
    else:
        print("Error fetching data from Open-Meteo API: " + str(response.status_code))

In [9]:
map_center = [20.0, 0.0]
m = LeafletMap(center=map_center, zoom=2, scroll_wheel_zoom=True)

draw_control = DrawControl(
    marker={'shapeOptions': {}},
    polyline={},
    polygon={},
    circle={},
    circlemarker={},
)
m.add_control(draw_control)
draw_control.on_draw(handle_marker)

# Display map
display(m)

Map(center=[20.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

[[{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}], [{'summary_text': '2024-11-03'}]]
